In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import OneHotEncoder

df_treino =  pd.read_csv('train.csv')
df_teste = pd.read_csv('test.csv')

var_categorica = df_treino.select_dtypes(include=['object']).columns
var_numerica = df_teste._get_numeric_data().columns

dict_nulls = {}
for i in df_treino.columns:
    dict_nulls[i] = df_treino[i].isnull().sum()

df_treino_null =  pd.DataFrame(data=list(dict_nulls.values()), index=list(dict_nulls.keys()), columns=['nulls'])

for i in var_categorica:
    if i != "Name":
        df_treino[i] = df_treino[i].fillna(df_treino[i].mode()[0])

for i in var_numerica:
    df_treino[i] = df_treino[i].fillna(df_treino[i].mean())

df_treino.describe()
df_treino.describe(include=['O'])
df_treino.drop(['Name'], axis=1, inplace=True)

df_treino['Deck'] = df_treino['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
df_treino['Port'] = df_treino['Cabin'].apply(lambda s: s[-1] if pd.notnull(s) else 'M')
df_treino["Deck"] = df_treino["Deck"].map({'B':0, 'F':1, 'A':2, 'G':3, 'E':4, 'D':5, 'C':6, 'T':7}).astype(int)
df_treino["Port"] = df_treino["Port"].map({'P':0, 'S':1}).astype(int)
df_treino.drop(['Cabin'], axis=1, inplace=True)
df_treino["HomePlanet"] = df_treino["HomePlanet"].map({'Earth':0, 'Europa':1, 'Mars':2}).astype(int)

if 'train' in 'train.csv':
    df_treino.drop(['PassengerId'], axis=1, inplace=True)

if 'VIP' in df_treino.columns:
    vip_unico = df_treino["VIP"].unique()
    df_treino["VIP"] = df_treino["VIP"].map(dict(zip(vip_unico, list(range(len(vip_unico)))))).astype(int)

if 'CryoSleep' in df_treino.columns:
    cryosleep_unico = df_treino["CryoSleep"].unique()
    df_treino["CryoSleep"] = df_treino["CryoSleep"].map(dict(zip(cryosleep_unico, list(range(len(cryosleep_unico)))))).astype(int)

if 'Destination' in df_treino.columns:
    destination_unico = df_treino["Destination"].unique()
    df_treino["Destination"] = df_treino["Destination"].map(dict(zip(destination_unico, list(range(len(destination_unico)))))).astype(int)

X = df_treino.drop("Transported", axis=1)
y = df_treino["Transported"]

var_categorica2 = X.select_dtypes(include='object').columns
codificador = OneHotEncoder(drop='first', sparse=False)
X_codificado = codificador.fit_transform(X[var_categorica2])
X_codificado = pd.DataFrame(X_codificado, columns=var_categorica2)
X = pd.concat([X.drop(var_categorica2, axis=1), X_codificado], axis=1)

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

lg = LogisticRegression()
lg.fit(X_treino, y_treino)
lg_score = lg.score(X_teste, y_teste)

rf = RandomForestClassifier()
rf.fit(X_treino, y_treino)
rf_score = rf.score(X_teste, y_teste)

gb = GradientBoostingClassifier()
gb.fit(X_treino, y_treino)
gb_score = gb.score(X_teste, y_teste)

resultados = pd.DataFrame(dict(model=['Logistic Regression', 'Random Forest', 'Gradient Boosting'],accuracy=[lg_score,rf_score, gb_score]))

resultados

c:\Users\fonsecamatheus\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\fonsecamatheus\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,model,accuracy
0,Logistic Regression,0.775733
1,Random Forest,0.785509
2,Gradient Boosting,0.785509
